In [2]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd

In [5]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return "Missing"
    result = re.search(r"published (\b\d+)", raw_year)
    if result == None:
        return "None"
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return "Missing"
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return "None"
    else:
        return result.group(1)

# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return "Missing"



In [7]:
books = []
for page in range(1, 25):
    response = requests.get("https://www.goodreads.com/shelf/show/horror?page=" + str(page))
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('div', class_ = "elementList")
    for book in book_container:
        bookTitle = solve_nonetype_error(book.find('a', class_ = "bookTitle"))
       
        bookAuthor =solve_nonetype_error(book.find('a', class_ = "authorName"))

        # After(use comments)
        year = clean_publish_year(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        rating = clean_rating(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        raters = clean_raters(solve_nonetype_error(book.find('span', class_ = "greyText smallText"))).replace(",", "")
        # # readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
        
        # Before (use comments)
        # year = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # rating = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # raters = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # readerNum = book.find('span', class_ = "greyText statistic").text.strip()


        book_data = [bookTitle, bookAuthor, year, rating, raters]
        books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating", "Raters"])
books.head()

,Title,Author,Year,Rating,Raters
0,The Shining (Paperback),Stephen King,1977,4.25,1328738
1,It (Paperback),Stephen King,1986,4.25,976857
2,Dracula (Paperback),Bram Stoker,1897,4.01,1116505
3,Carrie (Mass Market Paperback),Stephen King,1974,3.98,628585
4,Pet Sematary (Kindle Edition),Stephen King,1983,4.04,514640


In [8]:
books.to_csv("HorrorAfter.csv", index = False)